### Imports

In [1]:
import os
import pandas as pd
import requests
import yaml

from zipfile import ZipFile

### Functions

In [15]:
def load_config():
    """
    """

    config = yaml.safe_load(open('config.yaml'))

    return config

def download_content(url, file_path):
    """
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5'
    }

    response = requests.get(url, headers=headers)
    
    if response == 200:
        with open(file_path, 'wb') as f:
            f.write(response.content)
    else:
        print(response.status_code)

def extract_all(file_path):
    """
    """
    
    with ZipFile(file_path, 'r') as zf:
        zf.extractall()

def load_file(file_path, **kwargs):
    """
    """

    _, extsn = os.path.splitext(file_path)

    if extsn == '.csv':
        df = pd.read_csv(file_path, **kwargs)
    elif extsn == '.xlsx':
        df = pd.read_excel(file_path, **kwargs)
    
    return df

def download_ce_zip_file():
    """
    """
    
    config = load_config()

    url = os.path.join(
        config['BASE_URL'],
        config['SURVEY_SOURCE'] + config['COLLECTION_QUARTER'][2:4] + '.zip'
    )

    file_path = os.path.join(
        config['DATA_DIRECTORY'],
        config['SURVEY_SOURCE'] + config['COLLECTION_QUARTER'][2:4] + '.zip'
    )

    download_content(url=url, file_path=file_path)
    extract_all(file_path=file_path)

def load_ce_data():
    """
    """

    config = load_config()

    file_path = os.path.join(
        config['DATA_DIRECTORY'],
        config['SURVEY_SOURCE'].lower() + config['COLLECTION_QUARTER'][2:4],
        config['DATASET_BASE_NAME'] + config['COLLECTION_QUARTER'][2:] + config['DATASET_EXTENSION']
    )

    data = load_file(file_path=file_path)

    if all([col in data.columns for col in config['DATASET_COLUMNS']]):
        data = data[config['DATASET_COLUMNS']]

    return data

def load_ce_data_dict(sheet_name):
    """
    """

    # Load config.
    config = load_config()

    # Set file path for data dictionary.
    file_path = os.path.join(
        config['DATA_DIRECTORY'],
        config['DATA_DICTIONARY']
    )

    # Load file.
    data_dict = load_file(file_path=file_path, sheet_name=sheet_name)

    # Edit for data inconsistency between data files and data dictionary
    # files.
    if config['SURVEY_SOURCE'] == 'intrvw':
        survey_source = 'INTERVIEW'
    else:
        survey_source = config['SURVEY_SOURCE']

    # Subset the data diciontary to the survey source and dataset file
    # name.
    data_dict = data_dict.loc[
        (data_dict['Survey'] == survey_source)
        & (data_dict['File'] == config['DATASET_BASE_NAME'].upper())
        & (data_dict['Last quarter'].isna())
    ]

    # If variables descriptions tab, then keep variable name and
    # description. If looking for descriptions of the values, then
    # keep variable, value, and the value's description.
    if sheet_name == 'Variables':
        data_dict = data_dict.loc[
            data_dict['Variable Name'].isin(config['DATASET_COLUMNS']),
            ['Variable Name', 'Variable description']
        ]
    elif sheet_name == 'Codes ':
        data_dict = data_dict.loc[
            data_dict['Variable '].isin(config['DATASET_COLUMNS']),
            ['Variable ', 'Code value', 'Code description']
        ]

    return data_dict

def write_data_list(file_name, data_list):
    """
    """

    # Load config.
    config = load_config()

    # Create file path for writing data.
    file_path = os.path.join(config['DATA_DIRECTORY'], file_name)

    # For each data frame in the list of data frames, write to a sheet
    # on the workbook.
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for data, sheet_name in data_list:
            data.to_excel(writer, sheet_name=sheet_name, index=False)

### Download Data

In [16]:
# MANUAL DOWNLOAD REQUIRED.
# 
# REQUEST IS RETURNING RESPONSE 403. NOT SURE WHY BLS IS STRICT ABOUT
# DOWNLOADING PUMD PROGRAMTICALLY.
# 
# TODO: TRY CHANGING USER AGENT HEADER; SELENIUM WEB DRIVER; USING API
# (NOTORIOUS).

# download_ce_zip_file()

### Read Data

In [17]:
data = load_ce_data()
print(data.shape)

data_dict_vars = load_ce_data_dict(sheet_name='Variables')
print(data_dict_vars.shape)

data_dict_codes = load_ce_data_dict(sheet_name='Codes ')
print(data_dict_codes.shape)

(4751, 26)
(26, 2)
(109, 3)


### Write Data

In [18]:
file_name = 'ce_survey_data.xlsx'

df_list = [
    (data, 'data'),
    (data_dict_vars, 'data_var_desc'),
    (data_dict_codes, 'data_code_desc')
]

write_data_list(file_name=file_name, data_list=df_list)